# 张量计算

## 定义张量

In [1]:
import torch

In [2]:
x= torch.rand(5,3)
x

tensor([[0.8292, 0.9438, 0.8256],
        [0.3667, 0.6964, 0.4223],
        [0.3804, 0.2910, 0.5408],
        [0.8702, 0.4501, 0.3714],
        [0.0172, 0.6330, 0.1158]])

## 访问张量
张量的下标从0开始

In [3]:
#索引访问
x[1,2]

tensor(0.4223)

In [4]:
#切片访问
x[:,2]

tensor([0.8256, 0.4223, 0.5408, 0.3714, 0.1158])

## 张量运算

In [5]:
y= torch.rand(5,3)
q=x.mm(y.t())
q
# mm refers to matrix multiply
# .t() transform the y matrix from 5*3 to 3*5

tensor([[0.3563, 1.2247, 0.9368, 0.8686, 1.5348],
        [0.2342, 0.6012, 0.4930, 0.4908, 0.8061],
        [0.1255, 0.5973, 0.5102, 0.4113, 0.8121],
        [0.2236, 1.0376, 0.5885, 0.5704, 1.0169],
        [0.1757, 0.1471, 0.1754, 0.2429, 0.2873]])

## 张量与数组之间的转换

In [6]:
import numpy as np
x_tensor=torch.randn(2,3)
y_numpy=np.random.randn(2,3)
x_tensor,y_numpy

(tensor([[ 0.3642,  1.2540,  1.0739],
         [ 0.2917,  1.7837, -1.4569]]),
 array([[1.77638055, 0.23200153, 0.62601601],
        [0.19598787, 2.00028063, 0.38753166]]))

In [7]:
x_numpy=x_tensor.numpy()
y_tensor=torch.from_numpy(y_numpy)
x_numpy,y_tensor

(array([[ 0.36421117,  1.2539915 ,  1.0738758 ],
        [ 0.29165778,  1.7836677 , -1.4568555 ]], dtype=float32),
 tensor([[1.7764, 0.2320, 0.6260],
         [0.1960, 2.0003, 0.3875]], dtype=torch.float64))

## GPU上的张量计算

In [8]:
#验证GPU
torch.cuda.is_available()

True

In [9]:
#使用 GPU 
if torch.cuda.is_available():
    x=x.cuda()
    y=y.cuda()
    print(x+y)

tensor([[0.9424, 1.2172, 0.8309],
        [1.3381, 0.8395, 0.7665],
        [0.7086, 0.4439, 1.1710],
        [1.2156, 0.7604, 0.7219],
        [0.6599, 0.9042, 1.0194]], device='cuda:0')


In [10]:
#使用CPU
x=x.cpu()

# 动态计算图

## 自动微分变量
三个属性：
data：伴随自动微分变量的张量，专门储存计算结果
grad：.backward() 计算变量的梯度信息，并将叶节点的导数值储存在.grad中
grad_fn：回溯箭头

In [11]:
#动态计算图实例
x=torch.ones(2,2,requires_grad=True)
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [12]:
y=x+2
y.grad_fn

In [13]:
z=y*y
# * 是对应元素相乘的运算方法
z.grad_fn
#计算图每增加一次计算，就将相邻的连个计算节点连接在一起

In [14]:
t=torch.mean(z)

## 自动微分与导数计算
函数 t(x)=m(x+2)^2

In [15]:
t.backward()

In [16]:
print(z.grad)
print(y.grad)
print(x.grad)
#只有计算图中的叶节点才可以通过.backward()获得梯度信息

None
None
tensor([[1.5000, 1.5000],
        [1.5000, 1.5000]])


D:\anaconda_install\lib\site-packages\torch\_tensor.py:1104: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\build\aten\src\ATen/core/TensorBody.h:475.)
  return self._grad


In [17]:
#出现了多次的节点，计算梯度时会为每一个节点计算梯度信息，之后再把同样命名的节点的梯度累加起来
s=torch.tensor([[0.01,0.02]],requires_grad=True)
x=torch.ones(2,2,requires_grad=True)
for i in range(10):
    s=s.mm(x)
z=torch.mean(s)
z.backward()
print(s.grad)
print(x.grad)

None
tensor([[37.1200, 37.1200],
        [39.6800, 39.6800]])


# Pytorch 实例：预测房价

## 准备数据

In [18]:
#data
x= torch.linspace(0,100,100).type(torch.FloatTensor)

In [19]:
x

tensor([  0.0000,   1.0101,   2.0202,   3.0303,   4.0404,   5.0505,   6.0606,
          7.0707,   8.0808,   9.0909,  10.1010,  11.1111,  12.1212,  13.1313,
         14.1414,  15.1515,  16.1616,  17.1717,  18.1818,  19.1919,  20.2020,
         21.2121,  22.2222,  23.2323,  24.2424,  25.2525,  26.2626,  27.2727,
         28.2828,  29.2929,  30.3030,  31.3131,  32.3232,  33.3333,  34.3434,
         35.3535,  36.3636,  37.3737,  38.3838,  39.3939,  40.4040,  41.4141,
         42.4242,  43.4343,  44.4444,  45.4545,  46.4646,  47.4747,  48.4848,
         49.4949,  50.5051,  51.5152,  52.5253,  53.5354,  54.5455,  55.5556,
         56.5657,  57.5758,  58.5859,  59.5960,  60.6061,  61.6162,  62.6263,
         63.6364,  64.6465,  65.6566,  66.6667,  67.6768,  68.6869,  69.6970,
         70.7071,  71.7172,  72.7273,  73.7374,  74.7475,  75.7576,  76.7677,
         77.7778,  78.7879,  79.7980,  80.8081,  81.8182,  82.8283,  83.8384,
         84.8485,  85.8586,  86.8687,  87.8788,  88.8889,  89.89

In [20]:
rand= torch.randn(100)*10
y=x+rand

In [21]:
y

tensor([  5.8172,  18.6104,  -1.9580,   8.5302,  11.8980,  14.6372,   9.6438,
          8.4861,  -4.3653,  -9.1905,   4.2438,  12.3360,  25.1932,  27.9038,
         12.9774,  28.1908,  21.3552,  12.9696,  19.4254,  26.3623,  30.1259,
          2.3303,  28.0269,  29.0236,  31.8357,  39.6460,  24.4980,  24.5201,
         21.5651,  31.9815,  28.2859,  43.4920,  11.5212,  19.3245,  33.7715,
         34.3016,  46.1133,  46.2549,  25.8882,  32.3769,  55.2786,  27.1265,
         55.2514,  34.4277,  33.0483,  31.9809,  36.3990,  58.2681,  56.8617,
         38.6540,  35.8447,  35.5562,  57.1524,  51.0880,  37.5827,  48.4261,
         68.6381,  59.0323,  46.3818,  54.4188,  71.7219,  74.0232,  65.7253,
         57.0199,  49.7021,  64.9529,  68.7409,  63.8723,  46.1819,  65.7656,
         83.4676,  87.6298,  64.9460,  84.5302,  75.7578,  69.5401,  82.0338,
         77.2654,  77.4595,  77.4200,  88.4672,  79.3108,  68.0409,  90.8907,
        100.8298,  76.0059,  74.3922,  84.2244,  96.0150, 101.62

In [22]:
x_train=x[:-10]
x_test=x[-10 :]
y_train=y[:-10]
y_test=y[-10 :]

In [23]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10,8))
# plt.plot(x_train.data.numpy(),y_train.data.numpy(),'o')
# plt.xlabel('x')
# plt.ylabel('y')
# plt.show()

## 训练

In [24]:
a= torch.rand(1,requires_grad=True)
b= torch.rand(1,requires_grad=True)
learning_rate=0.0001

In [25]:
for i in range(1000):
    predictions= a.expand_as(x_train)*x_train+b.expand_as(x_train)
    loss=torch.mean((predictions-y_train)**2)
    print('loss:',loss)
    loss.backward()
    a.data.add_(-learning_rate*a.grad.data)
    b.data.add_(-learning_rate*b.grad.data)
    a.grad.data.zero_()
    b.grad.data.zero_()

loss: tensor(989.6157, grad_fn=<MeanBackward0>)
loss: tensor(286.9861, grad_fn=<MeanBackward0>)
loss: tensor(139.5769, grad_fn=<MeanBackward0>)
loss: tensor(108.6509, grad_fn=<MeanBackward0>)
loss: tensor(102.1627, grad_fn=<MeanBackward0>)
loss: tensor(100.8015, grad_fn=<MeanBackward0>)
loss: tensor(100.5159, grad_fn=<MeanBackward0>)
loss: tensor(100.4560, grad_fn=<MeanBackward0>)
loss: tensor(100.4434, grad_fn=<MeanBackward0>)
loss: tensor(100.4407, grad_fn=<MeanBackward0>)
loss: tensor(100.4401, grad_fn=<MeanBackward0>)
loss: tensor(100.4399, grad_fn=<MeanBackward0>)
loss: tensor(100.4398, grad_fn=<MeanBackward0>)
loss: tensor(100.4398, grad_fn=<MeanBackward0>)
loss: tensor(100.4398, grad_fn=<MeanBackward0>)
loss: tensor(100.4397, grad_fn=<MeanBackward0>)
loss: tensor(100.4397, grad_fn=<MeanBackward0>)
loss: tensor(100.4397, grad_fn=<MeanBackward0>)
loss: tensor(100.4396, grad_fn=<MeanBackward0>)
loss: tensor(100.4396, grad_fn=<MeanBackward0>)
loss: tensor(100.4395, grad_fn=<MeanBack

loss: tensor(100.4187, grad_fn=<MeanBackward0>)
loss: tensor(100.4186, grad_fn=<MeanBackward0>)
loss: tensor(100.4186, grad_fn=<MeanBackward0>)
loss: tensor(100.4185, grad_fn=<MeanBackward0>)
loss: tensor(100.4185, grad_fn=<MeanBackward0>)
loss: tensor(100.4185, grad_fn=<MeanBackward0>)
loss: tensor(100.4184, grad_fn=<MeanBackward0>)
loss: tensor(100.4184, grad_fn=<MeanBackward0>)
loss: tensor(100.4184, grad_fn=<MeanBackward0>)
loss: tensor(100.4183, grad_fn=<MeanBackward0>)
loss: tensor(100.4183, grad_fn=<MeanBackward0>)
loss: tensor(100.4183, grad_fn=<MeanBackward0>)
loss: tensor(100.4182, grad_fn=<MeanBackward0>)
loss: tensor(100.4182, grad_fn=<MeanBackward0>)
loss: tensor(100.4181, grad_fn=<MeanBackward0>)
loss: tensor(100.4181, grad_fn=<MeanBackward0>)
loss: tensor(100.4181, grad_fn=<MeanBackward0>)
loss: tensor(100.4180, grad_fn=<MeanBackward0>)
loss: tensor(100.4180, grad_fn=<MeanBackward0>)
loss: tensor(100.4179, grad_fn=<MeanBackward0>)
loss: tensor(100.4179, grad_fn=<MeanBack

loss: tensor(100.4117, grad_fn=<MeanBackward0>)
loss: tensor(100.4116, grad_fn=<MeanBackward0>)
loss: tensor(100.4116, grad_fn=<MeanBackward0>)
loss: tensor(100.4116, grad_fn=<MeanBackward0>)
loss: tensor(100.4115, grad_fn=<MeanBackward0>)
loss: tensor(100.4115, grad_fn=<MeanBackward0>)
loss: tensor(100.4114, grad_fn=<MeanBackward0>)
loss: tensor(100.4114, grad_fn=<MeanBackward0>)
loss: tensor(100.4113, grad_fn=<MeanBackward0>)
loss: tensor(100.4113, grad_fn=<MeanBackward0>)
loss: tensor(100.4113, grad_fn=<MeanBackward0>)
loss: tensor(100.4112, grad_fn=<MeanBackward0>)
loss: tensor(100.4112, grad_fn=<MeanBackward0>)
loss: tensor(100.4112, grad_fn=<MeanBackward0>)
loss: tensor(100.4111, grad_fn=<MeanBackward0>)
loss: tensor(100.4111, grad_fn=<MeanBackward0>)
loss: tensor(100.4110, grad_fn=<MeanBackward0>)
loss: tensor(100.4110, grad_fn=<MeanBackward0>)
loss: tensor(100.4110, grad_fn=<MeanBackward0>)
loss: tensor(100.4109, grad_fn=<MeanBackward0>)
loss: tensor(100.4109, grad_fn=<MeanBack

### 上述步骤技术细节
计算predictions时，对a，b进行了扩充维度
只能对自动微分变量进行数值更新，所以更新a.data
某个函数后面增加了_，表明要用这个函数额计算结果替换当前的变量

In [26]:
# import matplotlib.pyplot as plt
# x_data=x_train.data.numpy()
# plt.figure()
# xplot,=plt.plot(x_data,y_train.data.numpy())
# yplot =plt.plot(x_data,a.data.numpy()*x_data+b.data.numpy())

## 预测

In [27]:
predictions=a.expand_as(x_test)*x_test+b.expand_as(x_test)
predictions

tensor([89.4654, 90.4550, 91.4446, 92.4342, 93.4238, 94.4134, 95.4031, 96.3927,
        97.3823, 98.3719], grad_fn=<AddBackward0>)